In [1]:
!pip install tensorflow-datasets

In [2]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\affinity\anaconda3\lib\site-packages (19.3.1)


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import load_model
from datetime import datetime
#tf.enable_eager_execution()

Using TensorFlow backend.


In [4]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),    
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 512)         1

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).batch(64) # 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [8]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_accuracy',
    save_best_only=True)

In [9]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [10]:
history = model.fit(train_data,
                    epochs=24,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 17:23:25.783378: {'loss': 2.1964595123291017, 'accuracy': 0.465625, 'val_loss': 1.632261952017523, 'val_accuracy': 0.5005}
Epoch 1 ends at 17:24:06.301068: {'loss': 1.4876708639144898, 'accuracy': 0.60985, 'val_loss': 1.5233012422634538, 'val_accuracy': 0.5798}
Epoch 2 ends at 17:24:48.778383: {'loss': 1.214648219871521, 'accuracy': 0.664325, 'val_loss': 1.1766602465301563, 'val_accuracy': 0.6805}
Epoch 3 ends at 17:25:31.400199: {'loss': 0.9705186642169953, 'accuracy': 0.7169, 'val_loss': 1.1392768484771632, 'val_accuracy': 0.6809}
Epoch 4 ends at 17:26:13.572279: {'loss': 0.784703432559967, 'accuracy': 0.759875, 'val_loss': 1.1141847706144783, 'val_accuracy': 0.6688}
Epoch 5 ends at 17:26:55.866015: {'loss': 0.6874556879281998, 'accuracy': 0.784625, 'val_loss': 2.6635124330308027, 'val_accuracy': 0.5465}
Epoch 6 ends at 17:27:37.487494: {'loss': 0.552667821264267, 'accuracy': 0.818475, 'val_loss': 0.9551012107900753, 'val_accuracy': 0.7296}
Epoch 7 ends at 17:28:19.45

In [11]:
model.evaluate(test_data)

157/157 [==============================].2636 - accuracy: 0.75 - 0s 58ms/step - loss: 1.2982 - accuracy: 0.75 - 0s 48ms/step - loss: 1.5514 - accuracy: 0.75 - 0s 43ms/step - loss: 1.4133 - accuracy: 0.75 - 0s 40ms/step - loss: 1.2891 - accuracy: 0.76 - 0s 38ms/step - loss: 1.2123 - accuracy: 0.78 - 0s 37ms/step - loss: 1.2636 - accuracy: 0.78 - 0s 35ms/step - loss: 1.3100 - accuracy: 0.76 - 0s 34ms/step - loss: 1.2713 - accuracy: 0.76 - 0s 34ms/step - loss: 1.3037 - accuracy: 0.75 - 0s 33ms/step - loss: 1.2212 - accuracy: 0.77 - 0s 33ms/step - loss: 1.2520 - accuracy: 0.76 - 0s 33ms/step - loss: 1.2464 - accuracy: 0.76 - 0s 33ms/step - loss: 1.2372 - accuracy: 0.76 - 0s 32ms/step - loss: 1.2661 - accuracy: 0.75 - 1s 32ms/step - loss: 1.3295 - accuracy: 0.75 - 1s 31ms/step - loss: 1.3394 - accuracy: 0.74 - 1s 31ms/step - loss: 1.3439 - accuracy: 0.74 - 1s 31ms/step - loss: 1.3175 - accuracy: 0.75 - 1s 31ms/step - loss: 1.2996 - accuracy: 0.75 - 1s 31ms/step - loss: 1.2620 - accuracy: 0.

[1.2367899247035858, 0.7676]

In [12]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))